<a href="https://colab.research.google.com/github/cipz/AlgoritmiAvanzati/blob/master/Homework2/exactHeldKarp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Held and Karp algorithm

In [462]:
!pip install ipython-autotime

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 4.34 s


In [463]:
import math
import numpy as np
# u, v: vertexes/coordinates
# t: WEIGHT_TYPE
def weight (u, v, t):
  if t == "EUC_2D":
    return round(math.sqrt(sum([(a - b) ** 2 for a, b in zip(u, v)])))
  else:
    PI = 3.141592
    deg_xu = int(u[0])
    min_xu = u[0] - deg_xu
    rad_xu = PI * (deg_xu + 5.0 * min_xu/ 3.0) / 180.0

    deg_yu = int(u[1])
    min_yu = u[1] - deg_yu
    rad_yu = PI * (deg_yu + 5.0 * min_yu/ 3.0) / 180.0

    deg_xv = int(v[0])
    min_xv = v[0] - deg_xv
    rad_xv= PI * (deg_xv + 5.0 * min_xv/ 3.0) / 180.0

    deg_yv = int(v[1])
    min_yv = v[1]- deg_yv
    rad_yv = PI * (deg_yv + 5.0 * min_yv/ 3.0) / 180.0

    RRR = 6378.388
    q1 = math.cos(rad_yu - rad_yv)
    q2 = math.cos(rad_xu - rad_xv)
    q3 = math.cos(rad_xu + rad_xv)
    return (int) (RRR * math.acos(0.5 * ((1.0 + q1) * q2 - (1.0 - q1) * q3)) + 1.0)

time: 35.7 ms


In [464]:
lines = open("burma14.tsp", "r").readlines()
print(lines)
n = int(lines[3].split()[1]) #.split()[0] # extract number of vertexes
t = lines[4].split()[1]

'''for line in lines:
  if line.split()[0] == 'DIMENSION:':
    n = line.split()[0]
  else if line.split()[0] == 'EDGE_WEIGHT_TYPE:'
    t = line.split()[0]
  print()'''

V = [] # array of vertexes/coordinates
for i in range(8, 8 + n):
  V.append((int(lines[i].split()[0]) - 1, [float(lines[i].split()[1]), float(lines[i].split()[2])])) #(i, [x_value, y_value])

print(V)

print(weight(V[0][1], V[1][1], t))
print(weight(V[0][1], V[2][1], t))
print(weight(V[1][1], V[2][1], t))
print(weight(V[0][1], V[3][1], t))
print(weight(V[1][1], V[3][1], t))
print(weight(V[2][1], V[3][1], t))

['NAME: burma14\n', 'TYPE: TSP\n', 'COMMENT: 14-Staedte in Burma (Zaw Win)\n', 'DIMENSION: 14\n', 'EDGE_WEIGHT_TYPE: GEO\n', 'EDGE_WEIGHT_FORMAT: FUNCTION \n', 'DISPLAY_DATA_TYPE: COORD_DISPLAY\n', 'NODE_COORD_SECTION\n', '   1  16.47       96.10\n', '   2  16.47       94.44\n', '   3  20.09       92.54\n', '   4  22.39       93.37\n', '   5  25.23       97.24\n', '   6  22.00       96.05\n', '   7  20.47       97.02\n', '   8  17.20       96.29\n', '   9  16.30       97.38\n', '  10  14.05       98.12\n', '  11  16.53       97.38\n', '  12  21.52       95.59\n', '  13  19.41       97.13\n', '  14  20.09       94.55\n', 'EOF\n', '\n', '\n', '\n']
[(0, [16.47, 96.1]), (1, [16.47, 94.44]), (2, [20.09, 92.54]), (3, [22.39, 93.37]), (4, [25.23, 97.24]), (5, [22.0, 96.05]), (6, [20.47, 97.02]), (7, [17.2, 96.29]), (8, [16.3, 97.38]), (9, [14.05, 98.12]), (10, [16.53, 97.38]), (11, [21.52, 95.59]), (12, [19.41, 97.13]), (13, [20.09, 94.55])]
153
510
422
706
664
289
time: 30.4 ms


In [465]:
# global values

subsets = []
subsets.append(V)
#phi = [[None] * 10] * 10 #[[]]
#d = [[None] * 10] * 10
d = np.zeros(shape=(n,1000000)) # 2^(n-1)
phi = np.zeros(shape=(n,1000000))

def get_subset_index (S):
  cont = 0
  for s in subsets:
    if s == S:
      return cont
    else:
      cont += 1
  return False


time: 84.2 ms


In [466]:
def HK_VISIT (v, S): # v: arrival vertex of S, S: subset of vertexes 
  S_index = get_subset_index(S)
  #print(S_index)
  if (S_index is not False) & (len(S) == 1) & (S[0][0] == v): # Caso base: la soluzione è il peso dell’arco {v, 0}
    #print("1 case")
    return weight(V[v][1], V[0][1], t)
  elif d[v,S_index] != 0: # Distanza già calcolata, ritorna il valore memorizzato
    #print("2 case")
    return d[v,S_index] 
  else:  # Caso ricorsivo: trova il minimo tra tutti i sottocammini
    #print("3 case")
    mindist = math.inf
    minprec = None
    temp = [i for i in S if i[0] != v] # S \ {v} 
    subsets.append(temp) if temp not in subsets else subsets
    for u in temp:
      dist = HK_VISIT(u[0], temp)
      if (dist + weight(u[1], V[v][1], t)) < mindist:
        mindist = dist + weight(u[1], V[v][1], t)
        minprec = u[0]
    #print(v, S_index)
    d[v,S_index] = mindist
    phi[v,S_index] = minprec
    return mindist

time: 15.6 ms


In [467]:
def HK_TSP (G):
  G = (V, E) # build graph
  d = 0 # initialize matrix d
  phi = 0 # initialize matrix phi
  return HK_VISIT(0, V)

time: 1.95 ms


In [468]:
print(HK_VISIT(0, V))
print(len(subsets))
#print(np.matrix(subsets))
#print(np.matrix(d))

3323.0
8192
time: 1min 37s
